In [1]:
#!pip install beautifulsoup4
#!pip install sqlite3
#!pip install sqlalchemy

# Imports

In [10]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np
import re
import sqlite3
from sqlalchemy import create_engine

# Collect Data

In [11]:
#parameters
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

#URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

#Request to URL
page = requests.get (url, headers=headers)

#Beautiful soup object
soup = BeautifulSoup (page.text, 'html.parser')

#produts data
products = soup.find ('ul', class_= 'products-listing small')
products_list = products.find_all ('article', class_ = 'hm-product-item')

#product id
products_id = [p.get('data-articlecode') for p in products_list]

#product category
products_category = [p.get('data-category') for p in products_list]

#products name
products_list = products.find_all ('a', class_ = 'link')
products_name = [p.get_text() for p in products_list]

#product price
products_list = products.find_all ('span', class_ = 'price regular')
products_price = [p.get_text() for p in products_list]

#dataframe
data = pd.DataFrame([products_id, products_category, products_name, products_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

#scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H: %M: %S')

# Data Collect by product

In [24]:
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

#empty dataframe
df_compositions = pd.DataFrame()

#unique features
aux = []

df_pattern = pd.DataFrame(columns=['Art. No.', 'Composition', 'Fit', 'Product Safety', 'Size'])

for i in range(len(data)):
    #API request
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    
    page = requests.get (url, headers=headers)
    
    #beautiful soup object
    soup = BeautifulSoup (page.text, 'html.parser')
    
    
    #color product-----------------------------------------------------------------------------------
    product_list = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
    product_color = [p.get('data-color') for p in product_list]
    
    #product id
    product_id = [p.get('data-articlecode') for p in product_list]
    
    df_color = pd.DataFrame([product_id, product_color]).T
    df_color.columns = ['product_id', 'product_color']
    
    for j in range(len(df_color)):
        #API request
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
    
        page = requests.get (url, headers=headers)
    
        #beautiful soup object
        soup = BeautifulSoup (page.text, 'html.parser')
        
        #product name
        product_name = soup.find_all('h1', class_ = 'primary product-item-headline')
        product_name = product_name[0].get_text()
        
        #product price
        product_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+\.?\d+', product_price[0].get_text())[0]

        #composition--------------------------------------------------------------------------
        product_composition_list = soup.find_all ('div', class_='pdp-description-list-item')
        product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

        #rename df
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        #delete first row
        df_composition = df_composition.iloc[1:].fillna(method='ffill')

        #remove pocket lining, lining, shell
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex=True)

        #garantee same size of attributes
        #df_composition = pd.concat([df_pattern, df_composition], axis=0)
        
        #rename columns
        #df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size']

        #kepp new column if it shows up
        #aux = aux + df_composition.columns.tolist()

        #merge data color + decomposition
        #df_composition = pd.merge(df_composition, df_color,  how='left', on='product_id')

        #all products
        #df_compositions = pd.concat([df_compositions, df_composition], axis=0)

        
#join showroom data + analysis
#df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
#df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

#scrapy datetime
#df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H: %M: %S')

0
0
1
2
3
4
5
1
0
1
2
3
4
5
6
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
5
0
1
2
3
6
0
1
2
3
4
5
6
7
0
1
2
3
4
5
8
0
1
2
3
4
5
6
9
0
1
2
3
4
5
6
10
0
1
2
3
4
5
6
11
0
1
2
3
4
5
12
0
1
2
3
4
5
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
14
0
1
2
3
4
5
6
15
0
1
2
3
4
5
16
0
1
2
3
17
0
1
2
3
18
0
1
2
19
0
1
2
20
0
1
2
21
0
1
2
3
4
5
22
0
1
2
23
0
1
2
3
4
5
6
7
8
9
10
24
0
1
2
25
0
1
2
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
27
0
1
2
3
4
28
0
1
2
3
29
0
30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
31
0
1
2
32
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
33
0
1
2
3
4
5
6
34
0
1
2
35
0
1
2
3


In [25]:
df_composition

,Fit,Composition,Art. No.
1,Relaxed fit,Cotton 100%,0986779005
2,Relaxed fit,"Polyester 65%, Cotton 35%",0986779005


In [18]:
df_pattern.shape

(0, 5)

# Cleaning

In [67]:
data = data_raw
#product id
data = data.dropna(subset=['product_id'])
data['product_id'] = data['product_id'].astype(int)

#product name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ','_').lower())

#product_price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '')).astype(float)

#scrapy datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d  %H: %M: %S')

#style id
data['style_id'] = data['style_id'].astype(int)

#color id
data['color_id'] = data['color_id'].astype(int)

#color name
data['product_color'] = data['product_color'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

#fit
data['fit'] = data['Fit'].apply(lambda x: x.replace(' ','_').lower() if pd.notnull(x) else x)

#size number
data['size_number'] = data['Size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

#size model
data['size_model'] = data['Size'].str.extract('(\d+/\\d+)')

#composition
data = data[~data['Composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['Composition'].str.contains('Lining:', na=False)]
data = data[~data['Composition'].str.contains('Shell:', na=False)]
data = data[~data['Composition'].str.contains('Pocket:', na=False)]

#drop duplicates
data = data.drop_duplicates(subset=['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'product_color', 'Fit'], keep='last')

#reset index
data = data.reset_index(drop=True)

#break composition by comma
df1 = data['Composition'].str.split(',', expand=True)

#cotton | polyester | elastane | elasterell
df_ref = pd.DataFrame(index=np.arange(len(data)),  columns=['cotton', 'polyester', 'elastane', 'elasterell'])

#cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

#polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True),1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

#elastane
df_elastane = df1.loc[df1[1].str.contains('Elastane', na=True),1]
df_elastane.name = 'elastane'

#combine elastane from both columns 1 and 2
df_elastane = df_elastane.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elastane'] = df_ref['elastane'].fillna('Elastane 0%')

#elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell-P', na=True),1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna('Elasterell-P 0%')

#join dfs
data = pd.concat([data, df_ref], axis=1)

#format composition
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['elastane'] = data['elastane'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#drop columns
#data['composition'] = data['Composition']
data = data.drop(columns=['Size', 'Fit', 'Composition'], axis=1)

#drop duplicates
data = data.drop_duplicates()

(2013, 14)

In [25]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

page = requests.get (url, headers=headers)
soup = BeautifulSoup (page.text, 'html.parser')

In [26]:
total_item = soup.find_all ('h2', class_='load-more-heading')[0].get('data-total')
total_item

page_number = np.round(int(total_item)/36)

url2 = url + "?page-size=" + str(int(page_number*36))

page = requests.get (url2, headers=headers)
soup = BeautifulSoup (page.text, 'html.parser')

In [27]:
products = soup.find ('ul', class_= 'products-listing small')
products_list = products.find_all ('article', class_ = 'hm-product-item')

#product id
products_id = [p.get('data-articlecode') for p in products_list]

#product category
products_category = [p.get('data-category') for p in products_list]

#products name
products_list = products.find_all ('a', class_ = 'link')
products_name = [p.get_text() for p in products_list]

#product price
products_list = products.find_all ('span', class_ = 'price regular')
products_price = [p.get_text() for p in products_list]

#dataframe
data = pd.DataFrame([products_id, products_category, products_name, products_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

#scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H: %M: %S')
data

,product_id,product_category,product_name,product_price,scrapy_datetime
0,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-16 21: 38: 03
1,0427159006,men_jeans_ripped,Trashed Skinny Jeans,$ 39.99,2021-09-16 21: 38: 03
2,0730863005,men_jeans_skinny,Skinny Jeans,$ 29.99,2021-09-16 21: 38: 03
3,0636207006,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-16 21: 38: 03
4,0636207010,men_jeans_slim,Slim Jeans,$ 19.99,2021-09-16 21: 38: 03
...,...,...,...,...,...
67,0952016003,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99,2021-09-16 21: 38: 03
68,0868115005,men_jeans_skinny,Skinny Cropped Jeans,$ 29.99,2021-09-16 21: 38: 03
69,0811993024,men_jeans_regular,Regular Jeans,$ 29.99,2021-09-16 21: 38: 03
70,0751994018,men_jeans_slim,Slim Jeans,$ 29.99,2021-09-16 21: 38: 03


In [69]:
data

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,product_color,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,0690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40,690449,22,light_denim_blue_trashed,skinny_fit,187,32/32,0.98,0.0,0.02,0.0
1,0690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40,690449,22,denim_blue,skinny_fit,187,32/32,0.98,0.0,0.02,0.0
2,0690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40,690449,22,light_denim_blue,skinny_fit,187,32/32,0.98,0.0,0.02,0.0
3,0690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40,690449,22,black_washed_out,skinny_fit,187,32/32,0.98,0.0,0.02,0.0
4,0690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40,690449,22,white,skinny_fit,187,32/32,0.98,0.0,0.02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0636207001,men_jeans_slim,slim_jeans,19.99,2021-09-16 21:11:40,636207,1,midnight_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
353,0636207001,men_jeans_slim,slim_jeans,19.99,2021-09-16 21:11:40,636207,1,dark_gray,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
354,0636207001,men_jeans_slim,slim_jeans,19.99,2021-09-16 21:11:40,636207,1,denim_blue,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0
355,0636207001,men_jeans_slim,slim_jeans,19.99,2021-09-16 21:11:40,636207,1,white,slim_fit,NaN,NaN,0.89,0.1,0.01,0.0


In [99]:
#create table
query_showroom_schema = """
    CREATE TABLE vitrine_hm(
    product_id             INTEGER,
    product_category       TEXT,
    product_name           TEXT,
    product_price          REAL,
    scrapy_datetime        TEXT,
    style_id               INTEGER,
    color_id               INTEGER,
    product_color          TEXT,
    fit                    TEXT,
    size_number            REAL,
    size_model             TEXT,
    cotton                 REAL,
    polyester              REAL,
    elastane               REAL,
    elasterell             REAL
    )
"""

In [100]:
#connect to dataset
conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()
conn.close()

In [101]:
#engine db
conn = create_engine('sqlite:///hm_db.sqlite', echo=False)

#create columns
query = """
    SELECT * FROM vitrine_hm
"""

#df = pd.read_sql_query(query,conn)
#df.head()

#insert data to table
data.to_sql('vitrine_hm', con=conn, if_exists='append', index=False)
#data

In [102]:
df = pd.read_sql_query(query,conn)
df.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,product_color,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40.000000,690449,22,light_denim_blue_trashed,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
1,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40.000000,690449,22,denim_blue,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
2,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40.000000,690449,22,light_denim_blue,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
3,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40.000000,690449,22,black_washed_out,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
4,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-16 21:11:40.000000,690449,22,white,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0


In [98]:
#drop table
query = """
    DROP TABLE vitrine_hm
    
"""

conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query)
conn.commit()
conn.close()